In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class LeftoverCoarseLabelFinal(dj.Computed):
    definition = """
    # Ensure that Vertex labels for pinky.LeftoverProofreadLabel match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.LeftoverProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.LeftoverProofreadLabel
    """
    
    key_source = pinky.LeftoverProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.LeftoverProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.ExcitatoryLeftoverMeshes & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [4]:
start = time.time()
LeftoverCoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346349495971
Segment 648518346349495971 vertex labels regenerated in: 0.3472557067871094 seconds.
648518346349496103
Segment 648518346349496103 vertex labels regenerated in: 0.27642321586608887 seconds.
648518346349496245
Segment 648518346349496245 vertex labels regenerated in: 0.42690253257751465 seconds.
648518346349496554
Segment 648518346349496554 vertex labels regenerated in: 0.32732582092285156 seconds.
648518346349496672
Segment 648518346349496672 vertex labels regenerated in: 0.33228611946105957 seconds.
648518346349496925
Segment 648518346349496925 vertex labels regenerated in: 0.4002189636230469 seconds.
648518346349497151
Segment 648518346349497151 vertex labels regenerated in: 0.21387076377868652 seconds.
648518346349497752
Segment 648518346349497752 vertex labels regenerated in: 0.2768290042877197 seconds.
648518346349498239
Segment 648518346349498239 vertex labels regenerated in: 1.1739835739135742 seconds.
648518346349498406
Segment 648518346349498406 vertex labels

In [5]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()